# Installating Julia/IJulia (version of 2019-08-25)

### Warning: as of 2019-08-25, the Julia installation is not supposed to support a move of Winpython library

Read also https://pyjulia.readthedocs.io/en/latest/installation.html#step-1-install-julia

Other intesting notes:
or https://discourse.julialang.org/t/using-jupyterlab/20595/2
or https://blog.jupyter.org/i-python-you-r-we-julia-baf064ca1fb6

### 1 - Downloading and Installing the right Julia binary in the right place

In [ ]:
!echo %winpydirbase%
!if not exist "%winpydirbase%\t\Julia" mkdir "%winpydirbase%\t\Julia"
!echo "%winpydirbase%\t\Julia"

Download the latest 64 bit portable version of Julia (1.9.3 at the time of writing) from https://julialang.org/downloads/ .

Please make sure you check the checksums.

Extract the zip file in %winpydirbase%\t\Julia

### 2 - Initialize Julia , IJulia, and make them link to winpython
From here on we assume that Julia has been extracted in %winpydirbase%\t\Julia

In [ ]:
# will be in env next time
import os
os.environ["JULIA_HOME"] = os.environ["WINPYDIR"]+"\\..\\t\\Julia\\bin\\"
os.environ["JULIA_EXE"]="julia.exe"
os.environ["JULIA"]=os.environ["JULIA_HOME"]+os.environ["JULIA_EXE"]
os.environ["JULIA_PKGDIR"]=os.environ["WINPYDIRBASE"]+"\\settings\\.julia"

In [ ]:
# connecting Julia to WinPython (only once, or everytime you move things)
import julia
julia.install()

In [ ]:
%load_ext julia.magic

In [ ]:
%%julia  
import Pkg;
Pkg.add("PyPlot")
Pkg.add("Interact")
Pkg.add("Compose")
Pkg.add("SymPy")
Pkg.add("JuMP")
Pkg.add("Ipopt")
Pkg.add("IJulia")

### 3 - Launching a Julia Notebook 
Choose a Julia Kernel from Notebook, or Julia from Jupyterlab Launcher

### 4 - Julia Magic 
or use %load_ext julia.magic then %julia or %%julia

In [ ]:
import julia
%matplotlib inline
%load_ext julia.magic

In [ ]:
# since Julia 1.x ''@pyimport foo' is replaced per 'foo = pyimport("foo")'' 
%julia plt = pyimport("matplotlib.pyplot")
%julia np = pyimport("numpy")

In [ ]:
%%julia                                        
t = np.linspace(0, 2*pi,1000);             
s = np.sin(3*t + 4*np.cos(2*t));           
fig = plt.gcf()                         
plt.plot(t, s, color="red", linewidth=2.0, linestyle="--", label="sin(3t+4.cos(2t))")

In [ ]:
from julia import Main as jl
print(jl.eval("1+31"))
print(jl.eval("sqrt(1+31)"))